In [57]:
import random
import numpy as np

def generate_throws(number_of_throws):
    dice_options, dice_numbers = ['u', 'n'], [1, 2, 3, 4, 5, 6]
    result_numbers = []
    result_options = []
    dice = 'u'
    for i in range(number_of_throws):
        # losuję prawdopodobieństwo zmiany kości
        if dice == 'u':
            dice = random.choices(dice_options, weights=(96, 4), k=1)[0]
        if dice == 'n':
            dice = random.choices(dice_options, weights=(5, 95), k=1)[0]
        result_options.append(dice)
        # losuję rzut kością
        if dice == 'u':
            result_numbers.append(random.choices(dice_numbers, weights=(16.6, 16.6, 16.6, 16.6, 16.6, 16.6), k=1)[0])
        if dice == 'n':
            result_numbers.append(random.choices(dice_numbers, weights=(10, 10, 10, 10, 10, 50), k=1)[0])

    return result_options, result_numbers

In [58]:

def predict_throws_naive(throws):
    result_tab = []
    for i in range(len(throws)):
        result_tab.append('u')
    for i in range(len(throws)):
        if i > 2 and i < len(throws) - 3:
            checking_tab = throws[i-3:i+3]
            counter = 0
            for throw in checking_tab:
                if throw == 6:
                    counter +=1
            if counter >= 4: # jeśli w przedziale 6 sprawdzanych rzutów są 4, bądź więcej szóstek zakładamy że rzuty były oszukane (sprawdzałem również z wartością 3 i 5 ale dla 4 najlepiej zgaduje)
                for j in range(i-3,i+3):
                    result_tab[j] = 'n'
    return result_tab

def naive_test(n):
    for i in range(n):
        true_options, test_numbers = generate_throws(10000)
        naive_predicted = predict_throws_naive(test_numbers)
        ppb_counter = 0
        for i in range(len(true_options)):
            if true_options[i] == naive_predicted[i]:
                ppb_counter += 1
        print("Poprawność predykcji naiwnej:", "{0:.0%}".format(ppb_counter/len(true_options)))

naive_test(10)



Poprawność predykcji naiwnej: 79%
Poprawność predykcji naiwnej: 78%
Poprawność predykcji naiwnej: 78%
Poprawność predykcji naiwnej: 79%
Poprawność predykcji naiwnej: 78%
Poprawność predykcji naiwnej: 77%
Poprawność predykcji naiwnej: 78%
Poprawność predykcji naiwnej: 79%
Poprawność predykcji naiwnej: 78%
Poprawność predykcji naiwnej: 78%


In [65]:
# Z algorytmu Forward-Backward liczę alfy i bety

def predict_throws_fb(throws):
    dice_option_probability = [[0.96, 0.04], [0.05, 0.95]]
    dice_number_probability = [[0.166, 0.166, 0.166, 0.166, 0.166, 0.166], [0.1, 0.1, 0.1, 0.1, 0.1, 0.5]]
    alphas = np.zeros((2, 10000))
    betas = np.zeros((2, 10000))
    constant = 5.44

    def forward(throws):
        alphas[0,0] = 1
        alphas[1,0] = 0
        for t in range(1, len(throws)):
            for j in range(2):
                for i in range(2):
                    alphas[j,t] += alphas[i, t-1] * dice_option_probability[i][j] * dice_number_probability[i][throws[t] - 1]
                alphas[j,t] *= constant
        
    def backward(throws):
        betas[0, len(throws) - 1] = 1
        betas[1, len(throws) - 1] = 1
        for t in range(len(throws) - 2, -1, -1):
            for j in range(2):
                for i in range(2):
                    betas[j,t] += betas[i, t+1] * dice_option_probability[i][j] * dice_number_probability[i][throws[t] - 1]
                betas[j,t] *= constant
        
    forward(throws)
    backward(throws)
    gamma = (alphas * betas) / np.sum(alphas * betas, axis=0)
    return np.array(['u', 'n'])[(np.max(gamma, axis=0) != gamma[0,:]).astype(int)]


def fb_test(n):
    for i in range(n):
        true_options, test_numbers = generate_throws(10000)
        fb_predicted = predict_throws_fb(test_numbers)
        ppb_counter = 0
        for i in range(len(true_options)):
            if true_options[i] == fb_predicted[i]:
                ppb_counter += 1
        print("Poprawność predykcji z użyciem algorytmu FB:", "{0:.0%}".format(ppb_counter/len(true_options)))
    
fb_test(10)




Poprawność predykcji z użyciem algorytmu FB: 85%
Poprawność predykcji z użyciem algorytmu FB: 84%
Poprawność predykcji z użyciem algorytmu FB: 84%
Poprawność predykcji z użyciem algorytmu FB: 82%
Poprawność predykcji z użyciem algorytmu FB: 84%
Poprawność predykcji z użyciem algorytmu FB: 84%
Poprawność predykcji z użyciem algorytmu FB: 83%
Poprawność predykcji z użyciem algorytmu FB: 82%
Poprawność predykcji z użyciem algorytmu FB: 84%
Poprawność predykcji z użyciem algorytmu FB: 84%
